<a href="https://colab.research.google.com/github/swapnilmn/Assignment_03/blob/main/Question_1_Ass_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 (15 Marks)
Build a RNN based seq2seq model which contains the following layers: (i) input layer for character embeddings (ii) one encoder RNN which sequentially encodes the input character sequence (Latin) (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time (Devanagari). 


#Librares

In [ ]:
import csv
import numpy as np
import random
import torch
import torch.nn as nn 
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install wandb -qU
import wandb
wandb.login()
from google.colab import drive
drive.mount('/content/drive')

# Load of data

In [ ]:
import math  # Import the math module for mathematical operations
import time  # Import the time module for time-related functions
import torch  # Import the torch module for working with tensors
import numpy as np  # Import the numpy module for numerical operations

def asMinutes(s):
    # Convert seconds to minutes and seconds
    m = math.floor(s / 60)  # Calculate the number of minutes
    s -= m * 60  # Subtract the minutes converted to seconds from the total seconds
    return '%dm %ds' % (m, s)  # Return the formatted string for minutes and seconds

def span(since, percent):
    # Calculate the elapsed time and estimated remaining time
    now = time.time()  # Get the current time
    s = now - since  # Calculate the elapsed time in seconds
    es = s / (percent)  # Calculate the estimated total time in seconds
    rs = es - s  # Calculate the remaining time in seconds
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))  # Return the formatted string for elapsed time and remaining time

def key_presenting(dictionary, values):
    # Find keys in the dictionary that match the given values
    found_keys = []  # Initialize an empty list to store the found keys
    for value in values:  # Iterate over the given values
        for key, val in dictionary.items():  # Iterate over the dictionary items
            if value == val:  # If the value matches the dictionary value
                found_keys.append(key)  # Add the key to the list of found keys
    
    if found_keys:  # If any keys are found
        str = ''.join(found_keys)  # Concatenate the found keys into a single string
        if str[-1] == '\n':  # If the string ends with a newline character
            str = str[:-1]  # Remove the newline character from the end of the string
        elif str[0] == '\t':  # If the string starts with a tab character
            str = str[1:]  # Remove the tab character from the start of the string
    else:  # If no keys are found
        print("No keys found for the given values.")  # Print a message indicating no keys were found
    
    return str  # Return the string containing the found keys

def char_acc(targets, outputs):
    # Calculate character-level accuracy
    with torch.no_grad():  # Disable gradient calculation for efficiency
        count = 0  # Initialize a counter for correct predictions
        total_count = 0  # Initialize a counter for total predictions
        for i in range(targets.shape[0]):  # Iterate over the rows of the targets tensor
            same_elements = []  # Initialize an empty list to store the comparison results
            for j in range(targets.shape[1]):  # Iterate over the columns of the targets tensor
                if targets[i][j] != 67 or targets[i][j] != 66:  # Check if the target value is not equal to 67 or 66
                    same_elements.append(outputs[i][j].item() == targets[i][j].item())  # Append the comparison result to the list
            count += np.sum(same_elements)  # Add the number of True values in the list to the count
            total_count += len(same_elements)  # Add the total number of elements in the list to the total count
    return count / total_count  # Return the character-level accuracy as a ratio of correct predictions to total predictions

def word_acc(targets, outputs):
    # Calculate word-level accuracy
    outputs1 = torch.argmax(outputs, dim=1)  # Get the index of the maximum value along the second dimension of the outputs tensor
    with torch.no_grad():  # Disable gradient calculation for efficiency
        count = 0  # Initialize a counter for correct predictions
        for i in range(targets.shape[0]):  # Iterate over the rows of the targets tensor
            if (outputs1[i] == targets[i]).sum().item() == targets.shape[1]:  # Check if all elements in the row are equal
                count = count + 1  # Increment the count if all elements are equal
    return count / targets.shape[0]  # Return the word-level accuracy as a ratio of correct predictions to total predictions

def equi_points(data, epochs):
    # Sample equidistant points from data
    step = len(data) // epochs  # Calculate the step size to sample equidistant points
    indices = np.arange(0, len(data), step)  # Generate indices using numpy arange function
    equidistant_points = [data[i] for i in indices]  # Extract the equidistant points from the data list
    
    return equidistant_points  # Return the list of equidistant points


In [ ]:
import csv

def read_csv_files():
    # Define the file paths for the CSV files
    file_paths = {
        'file1': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_test.csv',  # Path to 'mar_test.csv'
        'file2': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_valid.csv',  # Path to 'mar_valid.csv'
        'file3': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv'  # Path to 'mar_train.csv'
    }

    data = {}  # Dictionary to store the CSV data

    for file_key, file_path in file_paths.items():
        with open(file_path) as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)  # Get the header row of the CSV
            rows = [row for row in csv_reader]  # Get all the data rows of the CSV
            data[file_key] = {
                'header': header,
                'rows': rows
            }

    return data

# Execute the function to read CSV files
csv_data = read_csv_files()

# Access the data
test = csv_data['file1']['rows']  # Contains rows from 'mar_test.csv'
val = csv_data['file2']['rows']  # Contains rows from 'mar_valid.csv'
train = csv_data['file3']['rows']  # Contains rows from 'mar_train.csv'


# Data Preprocessing

In [ ]:
# Define a function to read data from a list of pairs
def Data_Reading(list):
  i = []  # List to store the first element of each pair
  t = []  # List to store the second element of each pair
  for pair in list:
    i.append(pair[0])  # Append the first element to 'i'
    t.append(pair[1])  # Append the second element to 't'
  return i, t  # Return the two lists

# Read data from the 'train' list using the 'Data_Reading' function
train_inputs, train_targets = Data_Reading(train)

# Read data from the 'test' list using the 'Data_Reading' function
test_inputs, test_targets = Data_Reading(test)

# Read data from the 'val' list using the 'Data_Reading' function
val_inputs, val_targets = Data_Reading(val)

# Print the second element of the 'train_inputs' list
print(train_inputs[1])

# Print the second element of the 'train_targets' list
print(train_targets[1])

# Define a dictionary to store variable values
variable_dict = {
    'sc': '\t',    # 'sc' represents a tab character
    'eb': '\n',    # 'eb' represents a newline character
    'bc ': ' ',    # 'bc' represents a space character
    'unc': '\r'    # 'unc' represents a carriage return character
}

# Accessing the values from the dictionary
sc = variable_dict['sc']    # Assign the value of '\t' to 'sc'
eb = variable_dict['eb']    # Assign the value of '\n' to 'eb'
bc  = variable_dict['bc ']  # Assign the value of ' ' to 'bc'
unc = variable_dict['unc']  # Assign the value of '\r' to 'unc'


# Function to create dictionaries for language encoding

def dictlang(inputs, targets):
    dict = {}  # Dictionary for input language
    mil = 0  # Maximum input language length
    ichar = []  # List of characters in input language

    dict_t = {}  # Dictionary for target language
    mtl = 0  # Maximum target language length
    tchar = []  # List of characters in target language

    for s in inputs:
        mil = max(len(s), mil)
        for char in s:
            if char not in dict:
                dict[char] = len(ichar)
                ichar.append(char)

    # Add special characters if they are not present in the input language dictionary
    if bc not in dict:
        dict[bc] = len(ichar)
        ichar.append(bc)

    dict[unc] = len(ichar)
    ichar.append(unc)

    # Add special characters if they are not present in the target language dictionary
    if sc not in dict_t:
        dict_t[sc] = len(tchar)
        tchar.append(sc)

    for s in targets:
        mtl = max(len(s) + 2, mtl)
        for char in s:
            if char not in dict_t:
                dict_t[char] = len(tchar)
                tchar.append(char)

    # Add special characters if they are not present in the target language dictionary
    if eb not in dict_t:
        dict_t[eb] = len(tchar)
        tchar.append(eb)

    # Add special characters if they are not present in the target language dictionary
    if bc not in dict_t:
        dict_t[bc] = len(tchar)
        tchar.append(bc)

    return dict, mil, ichar, dict_t, mtl, tchar

# Create dictionaries for language encoding using training, validation, and test inputs and targets
dict, mil, ichar, dict_t, mtl, tchar = dictlang(train_inputs + val_inputs + test_inputs, train_targets + val_targets + test_targets)


def encoding_w(a, d, ml, l):
    encs = []  # List to store encoded words
    for word in a:
        enc = []  # List to store encoded characters of a word
        for char in word:
            if char in d:
                enc.append(d[char])  # Encode character if present in the dictionary
            else:
                enc.append(d[unc])  # Encode unknown character
        if (l == 0):
            while len(enc) < ml:
                enc.append(d[bc])  # Pad with blank character if required
        if (l == 1):
            enc.insert(0, d[sc])  # Insert start character at the beginning
            while len(enc) < ml - 1:
                enc.append(d[bc])  # Pad with blank character if required
            enc.append(d[eb])  # Append end character at the end
        encs.append(enc)  # Add encoded word to the list
    return encs

def tokenize(train,val,test,dict,dict_t,mil,mtl):
    # Tokenize the data for training, validation, and testing sets
    ti, tt = Data_Reading(train)  # Read training data
    tti, ttt = Data_Reading(test)  # Read testing data
    vi, vt = Data_Reading(val)  # Read validation data

    # Encode the training, validation, and testing data
    e_t_i = encoding_w(ti,dict,mil,0)  # Encode training input data
    e_t_t = encoding_w(tt,dict_t,mtl,1)  # Encode training target data
    e_v_i = encoding_w(vi,dict,mil,0)  # Encode validation input data
    e_v_t = encoding_w(vt,dict_t,mtl,1)  # Encode validation target data
    e_tt_i = encoding_w(tti,dict,mil,0)  # Encode testing input data
    e_tt_t = encoding_w(ttt,dict_t,mtl,1)  # Encode testing target data

    return e_t_i, e_t_t, e_v_i, e_v_t, e_tt_i, e_tt_t

# Tokenize the data using the given parameters
e_t_i, e_t_t, e_v_i, e_v_t, e_tt_i, e_tt_t = tokenize(train, val, test, dict, dict_t, mil, mtl)

# Generate a random number within the range [0, 100]
r = random.randint(0, 100)

# Print the keys corresponding to the values of the encoded data at index r
print(key_presenting(dict, e_t_i[int(r)]))
print(key_presenting(dict_t, e_t_t[int(r)]))


# Define a function to convert input and target data into tensor pairs
def tensor_conversion(ti, tg):
    pr = []
    for id, td in zip(ti, tg):
        # Convert each item in ti to a torch tensor
        it = torch.tensor(id)
        # Convert each item in tg to a torch tensor
        tt = torch.tensor(td)
        # Append the tensor pair to the result list
        pr.append((it, tt))
    return pr

# Convert e_t_i and e_t_t to tensor pairs
e_t_p = tensor_conversion(e_t_i, e_t_t)

# Convert e_v_i and e_v_t to tensor pairs
e_v_p = tensor_conversion(e_v_i, e_v_t)

# Re-assign e_t_p with new tensor pairs converted from e_tt_i and e_tt_t
e_t_p = tensor_conversion(e_tt_i, e_tt_t)

# Create a tuple containing e_t_p, e_v_p, and e_t_p
pairs = (e_t_p, e_v_p, e_t_p)

# Randomly choose a pair from e_t_p
pair = random.choice(e_t_p)

# Print the keys in dict corresponding to the values in pair[0]
print(key_presenting(dict, pair[0]))

# Print the keys in dict_t corresponding to the values in pair[1]
print(key_presenting(dict_t, pair[1]))
